# キャリブレーション


In [1]:
PROGRAM_NAME = "07_calibration_v2"

In [2]:
from quri_parts_oqtopus.backend import OqtopusSseBackend
from oqtopus_sse_pulse.adapter import QuriAdapter, collect_payloads_from_job, collect_session_from_job
from oqtopus_sse_pulse.plotters import plot_payload

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

DEVICE = "anemone" # デバイス名 (勉強会ではanemoneに固定)
backend = OqtopusSseBackend() # OqtopusSseBackendのインスタンスを作成

def experiment(program: str) -> list:
    """
    指定した実験プログラムを実行し, 測定結果のペイロードを返す関数.
    """

    # ジョブの実行
    job = backend.run_sse(
        file_path=f"program/{program}.py",
        device_id=f"{DEVICE}-pulse",
        name=program,
    )
    print(f"job_id = {job.job_id}")

    # ジョブの完了待ち
    job.wait_for_completion()
    print(f"job_status = {job.status}  elapsed={job.execution_time}")

    # QuriAdapterの作成
    adapter = QuriAdapter(backend)

    # 測定結果の取得
    payloads = collect_payloads_from_job(
        backend=adapter,
        job_id=job.job_id,
    )
    
    return payloads

In [9]:
payloads = experiment(PROGRAM_NAME)

job_id = 0696c87c-330d-7737-8000-4924d437f8de
job_status = succeeded  elapsed=1056.766


In [10]:
from pprint import pprint

print("=== Payload Status ===")
print(payloads[0]["status"])
print()

print("=== Payload Content ===")
pprint(payloads[0])

=== Payload Status ===
succeeded

=== Payload Content ===
{'calib_note': {'hpi_params': {'Q36': {'amplitude': 0.02610403040293455,
                                       'duration': 32.0,
                                       'target': 'Q36',
                                       'tau': 12,
                                       'timestamp': '2026-01-18 07:35:49'},
                               'Q37': {'amplitude': 0.03642034103201783,
                                       'duration': 32.0,
                                       'target': 'Q37',
                                       'tau': 12,
                                       'timestamp': '2026-01-18 07:35:57'},
                               'Q38': {'amplitude': 0.06933243739279625,
                                       'duration': 32.0,
                                       'target': 'Q38',
                                       'tau': 12,
                                       'timestamp': '2026-01-18 07:36:04'},
      

failedと表示されたらもう一度calibrationを行う。  
succeededなら次へ。

# calib_note, props, paramsの書き出し

以下のブロックでcalib_note, props, paramsの書き出しを行うので、必要な箇所を書き換えてpushする。

In [11]:
import json
import os
import yaml


if not os.path.exists("data"):
    os.makedirs("data")

with open(os.path.join("data", "calib_note.json"), "w") as f:
    json.dump(payloads[0].get("calib_note"), f, ensure_ascii=False, indent=2)
with open(os.path.join("data", "props.yaml"), "w", encoding="utf-8") as f:
    yaml.safe_dump(payloads[0].get("props"), f)
with open(os.path.join("data", "params.yaml"), "w", encoding="utf-8") as f:
    yaml.safe_dump(payloads[0].get("params"), f)
